# Ligand ADMET and Potency (Property Prediction)

The [ADMET](https://polarishub.io/competitions/asap-discovery/antiviral-admet-2025) and [Potency](https://polarishub.io/competitions/asap-discovery/antiviral-potency-2025) Challenge of the [ASAP Discovery competition](https://polarishub.io/blog/antiviral-competition) take the shape of a property prediction task. Given the SMILES (or, to be more precise, the CXSMILES) of a molecule, you are asked to predict the numerical properties of said molecule. This is a relatively straight-forward application of ML and this notebook will quickly get you up and running!

To begin with, choose one of the two challenges! The code will look the same for both. 

In [1]:
import polaris as po
from polaris.hub.client import PolarisHubClient
import csv

In [2]:
client = PolarisHubClient()
client.login()

[2025-02-07 15:27:21] INFO     You are successfully logged in to the Polaris Hub.                     ]8;id=951562;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/hub/client.py\client.py]8;;\:]8;id=182094;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/hub/client.py#252\252]8;;\

In [3]:
CHALLENGE = "antiviral-potency-2025"  # or: "antiviral-potency-2025"

## Load the competition

Let's first load the competition from Polaris.

Make sure you are logged in! If not, simply run `polaris login` and follow the instructions. 

In [4]:
import polaris as po

competition = po.load_competition(f"asap-discovery/{CHALLENGE}")

[2025-02-07 15:27:23] INFO     The version of Polaris that was used to create the artifact (0.11.0) ]8;id=802859;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=419865;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/_artifact.py#92\92]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.11.5).                                                                           

                      WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=841893;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=457841;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/dataset/_base.py#182\182]8;;\
                               enough, consider caching the dataset first using                                    
                               CompetitionSpecification.cache() for more performant data access.                   

As suggested in the logs, we'll cache the dataset. Note that this is not strictly necessary, but it does speed up later steps.

In [5]:
competition.cache()

Output()

[15:27:24] Copying to destination                                                                      ]8;id=120087;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=992640;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/dataset/_base.py#381\381]8;;\
           /home/valerij/.cache/polaris/datasets/d04266f9-7493-4fd1-b0a1-e6e9c22c251f/data.zarr                    

           For large Zarr archives, this may take a while.                                             ]8;id=653687;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=359047;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/dataset/_base.py#382\382]8;;\

[15:27:25]  Success: Copying Zarr keys                                                                ]8;id=565432;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=195310;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

            Success: Copying Zarr archive                                                             ]8;id=170197;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=351971;file:///home/valerij/miniconda3/envs/pol/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

'/home/valerij/.cache/polaris/datasets/d04266f9-7493-4fd1-b0a1-e6e9c22c251f'

Let's get the train and test set and take a look at the data structure.

In [6]:
train, test = competition.get_train_test_split()

In [7]:
train[10]

('CN1C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)C12CN(S(C)(=O)=O)C2',
 {'pIC50 (MERS-CoV Mpro)': 4.69, 'pIC50 (SARS-CoV-2 Mpro)': nan})

In [8]:
print(type(train))

<class 'polaris.dataset._subset.Subset'>


In [9]:
train[:1]

(array(['COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C |&1:3|'],
       dtype=object),
 {'pIC50 (MERS-CoV Mpro)': array([4.19]),
  'pIC50 (SARS-CoV-2 Mpro)': array([nan])})

In [10]:
import pandas as pd

smiles_list_test = []

# Iterate through train and populate the lists
for t in test:
    smiles_list_test.append(t)

# Create the DataFrame
potency_df_test = pd.DataFrame({
    'SMILES': smiles_list_test,
})

In [11]:
potency_df_test

,SMILES
0,C=CC(=O)NC1=CC=CC(N(CC2=CC=CC(Cl)=C2)C(=O)CC2=...
1,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...
2,CNC(=O)CN1C[C@]2(CCN(C3=CN=CC4=CC=C(OC[C@H](O)...
3,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...
4,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...
...,...
292,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...
293,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...
294,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H]2CCCC[C@@H]...
295,COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...


In [12]:
import pandas as pd
# Initialize lists to hold data for DataFrame
smiles_list = []
sars_list = []
mers_list = []

# Iterate through train and populate the lists
for t in train:
    smiles_list.append(t[0])
    sars_list.append(t[1].get('pIC50 (SARS-CoV-2 Mpro)', None))
    mers_list.append(t[1].get('pIC50 (MERS-CoV Mpro)', None))

# Create the DataFrame
potency_df_sars = pd.DataFrame({
    'SMILES': smiles_list,
    'pIC50 (SARS-CoV-2 Mpro)': sars_list,
})

# Create the DataFrame
potency_df_mers = pd.DataFrame({
    'SMILES': smiles_list,
    'pIC50 (MERS-CoV Mpro)': mers_list,
})

# Display the DataFrame
print(potency_df_sars)
print(potency_df_mers)

                                                 SMILES  \
0     COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...   
1     C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...   
2     CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...   
3     C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...   
4     C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...   
...                                                 ...   
1026  CNS(=O)(=O)OCC(=O)N1CCN(CC2=CC=CC(Cl)=C2)[C@H]...   
1027  O=C(CC1=CN=CC2=CC=CC=C12)N1CC[C@@H]2CCCC[C@H]2...   
1028  CNC(=O)[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)C1...   
1029  C[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)[C@H]1C ...   
1030  O=C(O)C[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)C1...   

      pIC50 (SARS-CoV-2 Mpro)  
0                         NaN  
1                        5.29  
2                         NaN  
3                        6.11  
4                        5.62  
...                       ...  
1026                     6.38  
1027                     6.09  
102

In [13]:
print(len(train))

1031


In [14]:
print(len(test))

297


### Raw data dump
We've decided to sacrifice the completeness of the scientific data to improve its ease of use. For those that are interested, you can also access the raw data dump that this dataset has been created from.

## Build a model
Next, we'll train a simple baseline model using scikit-learn. 

You'll notice that the challenge has multiple targets.

In [15]:
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer

In [16]:
mers_props = [
    {"name": "pIC50 (MERS-CoV Mpro)", "task": "REGRESSION"},]

## MERS

In [17]:
potency_df_mers

,SMILES,pIC50 (MERS-CoV Mpro)
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,4.19
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,4.92
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,4.73
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,4.90
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,4.81
...,...,...
1026,CNS(=O)(=O)OCC(=O)N1CCN(CC2=CC=CC(Cl)=C2)[C@H]...,5.57
1027,O=C(CC1=CN=CC2=CC=CC=C12)N1CC[C@@H]2CCCC[C@H]2...,4.60
1028,CNC(=O)[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)C1...,4.22
1029,C[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)[C@H]1C ...,4.40


In [18]:
dataset_mers = QSPRDataset(
    name="MultiTaskTutorialDataset",
    df=potency_df_mers,
    target_props=mers_props,
    store_dir="mers_output_adjusted/data",
    random_state=42,
)

dataset_mers.getDF()

qsprpred - WARNING - Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.


,SMILES,pIC50 (MERS-CoV Mpro),QSPRID,pIC50 (MERS-CoV Mpro)_original,Split_IsTrain
QSPRID,,,,,
MultiTaskTutorialDataset_0000,COC[C@]1(C)C(=O)N(c2cncc3ccccc23)C(=O)N1C,4.19,MultiTaskTutorialDataset_0000,4.19,True
MultiTaskTutorialDataset_0001,C=C(CN1CCc2ccc(Cl)cc2C1C(=O)Nc1cncc2ccccc12)C(...,4.92,MultiTaskTutorialDataset_0001,4.92,True
MultiTaskTutorialDataset_0002,CNC(=O)CN1C[C@]2(C[C@H](C)N(c3cnccc3C3CC3)C2=O...,4.73,MultiTaskTutorialDataset_0002,4.73,True
MultiTaskTutorialDataset_0003,C=C(CN1CCc2ccc(Cl)cc2C1C(=O)Nc1cncc2ccccc12)C(...,4.90,MultiTaskTutorialDataset_0003,4.90,True
MultiTaskTutorialDataset_0004,C=C(CN1CCc2ccc(Cl)cc2C1C(=O)Nc1cncc2ccccc12)C(...,4.81,MultiTaskTutorialDataset_0004,4.81,True
...,...,...,...,...,...
MultiTaskTutorialDataset_1026,CNS(=O)(=O)OCC(=O)N1CCN(Cc2cccc(Cl)c2)[C@H]2CS...,5.57,MultiTaskTutorialDataset_1026,5.57,True
MultiTaskTutorialDataset_1027,O=C(Cc1cncc2ccccc12)N1CC[C@@H]2CCCC[C@H]2C1,4.60,MultiTaskTutorialDataset_1027,4.60,True
MultiTaskTutorialDataset_1028,CNC(=O)[C@H]1CCCN(C(=O)Cc2cncc3ccccc23)C1,4.22,MultiTaskTutorialDataset_1028,4.22,True


In [19]:
from qsprpred.data import RandomSplit, BootstrapSplit, ScaffoldSplit, ClusterSplit
from qsprpred.data.descriptors.fingerprints import MorganFP, RDKitFP
from qsprpred.data.processing.feature_filters import LowVarianceFilter
from sklearn.preprocessing import StandardScaler

# Specifiy random split for creating the train (80%) and test set (20%)
rand_split = ClusterSplit(test_fraction=0.2, dataset=dataset_mers)
# rand_split = RandomSplit(test_fraction=0.15, dataset=dataset_mers)

# calculate compound features and split dataset into train and test
dataset_mers.prepareDataset(
    split=rand_split,
    feature_calculators=[MorganFP(radius=3, nBits=1024), RDKitFP(maxPath=7, nBits=512)],
    recalculate_features=True,
    feature_filters=[LowVarianceFilter(0.001)],
    feature_standardizer=StandardScaler()
)

print(f"Number of samples train set: {len(dataset_mers.y)}")
print(f"Number of samples test set: {len(dataset_mers.y_ind)}")

dataset_mers.save()

Number of samples train set: 721
Number of samples test set: 180


In [20]:
from qsprpred.models import SklearnModel
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from qsprpred.models import CrossValAssessor, TestSetAssessor

os.makedirs("class_models/XG_models_clust", exist_ok=True)

# This is an SKlearn model, so we will initialize it with the SklearnModel class
model_mers = SklearnModel(
    base_dir="class_models/XG_models_clust_mers_adjusted",
    alg=XGBRegressor,
    name="XGMorgan_clustsplit"
)

import numpy as np
from qsprpred.models import GridSearchOptimization, CrossValAssessor

In [21]:
from qsprpred.models import OptunaOptimization, TestSetAssessor
from sklearn.model_selection import KFold

# Note the specification of the hyperparameter types as first item in the list
search_space = {"max_depth": ["int", 1, 20],
                "gamma": ["int", 0, 20],
                "max_delta_step": ["int", 0, 20],
                "min_child_weight": ["int", 1, 20],
                "learning_rate": ["float", 0.001, 1],
                "subsample": ["float", 0.001, 1],
                "n_estimators": ["int", 10, 250],
               }

# Optuna gridsearcher with the TestSetAssessor
gridsearcher = OptunaOptimization(
    n_trials=400,
    param_grid=search_space,
    model_assessor=TestSetAssessor(scoring='r2'),
)
gridsearcher.optimize(model_mers, dataset_mers)

[I 2025-02-07 15:28:16,151] A new study created in memory with name: no-name-28c37ca0-c6df-4ffa-9435-4e648c0def1e
[I 2025-02-07 15:28:16,465] Trial 0 finished with value: 0.0395191437115342 and parameters: {'max_depth': 8, 'gamma': 19, 'max_delta_step': 15, 'min_child_weight': 12, 'learning_rate': 0.1568626218019941, 'subsample': 0.15683852581586644, 'n_estimators': 23}. Best is trial 0 with value: 0.0395191437115342.
[I 2025-02-07 15:28:16,624] Trial 1 finished with value: -0.203632929744739 and parameters: {'max_depth': 18, 'gamma': 12, 'max_delta_step': 14, 'min_child_weight': 1, 'learning_rate': 0.9699399423098324, 'subsample': 0.8326101981596213, 'n_estimators': 61}. Best is trial 0 with value: 0.0395191437115342.
[I 2025-02-07 15:28:16,920] Trial 2 finished with value: -0.08177786666915665 and parameters: {'max_depth': 4, 'gamma': 3, 'max_delta_step': 6, 'min_child_weight': 11, 'learning_rate': 0.43251307362347363, 'subsample': 0.2919379110578439, 'n_estimators': 157}. Best is tr

{'max_depth': 16,
 'gamma': 2,
 'max_delta_step': 4,
 'min_child_weight': 20,
 'learning_rate': 0.1673266351389645,
 'subsample': 0.17794938759715506,
 'n_estimators': 74}

{'max_depth': 16,
 'gamma': 2,
 'max_delta_step': 4,
 'min_child_weight': 20,
 'learning_rate': 0.1673266351389645,
 'subsample': 0.17794938759715506,
 'n_estimators': 74}

In [32]:
# We can now assess the model performance on the training set using cross validation
from sklearn.metrics import mean_squared_error
import sklearn
from sklearn import metrics

print(sklearn.metrics.get_scorer_names())

CrossValAssessor(
    scoring="neg_mean_absolute_error",
    split=BootstrapSplit(split=ClusterSplit(dataset_mers), n_bootstraps=100)
)(model_mers, dataset_mers)

# and on the test set
TestSetAssessor("neg_mean_absolute_error")(model_mers, dataset_mers)

# Finally, we need to fit the model on the complete dataset if we want to use it further
model_mers.fitDataset(dataset_mers)

# and save the model
_ = model_mers.save()

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc',

In [ ]:
from rdkit import Chem
from rdkit.Chem import MolStandardize

def read_smi_file(file_path):
    smiles_list = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Skip comment lines and empty lines
                if line.strip() and not line.startswith('#'):
                    # SMILES strings are typically separated by whitespace or tab
                    smiles = line.split()[0]
                    smiles_list.append(smiles)
    except FileNotFoundError:
        print("File not found.")
    return smiles_list

# Example usage to read the standardized SMILES file and create a list
file_path = 'test_potency.smi'
smiles_list = read_smi_file(file_path)
print("SMILES in the file:")
print(len(smiles_list))

In [33]:
well = model_mers.predictMols(smiles_list_test)

In [34]:
print(well)

[[5.320606 ]
 [5.2716703]
 [5.059407 ]
 [5.1448517]
 [5.284195 ]
 [5.2422705]
 [5.368682 ]
 [5.3890457]
 [5.09416  ]
 [5.0523124]
 [5.008095 ]
 [5.4924116]
 [5.317238 ]
 [5.3535748]
 [5.489856 ]
 [5.3856745]
 [5.1945477]
 [4.9230795]
 [4.5703797]
 [5.1945477]
 [5.310707 ]
 [5.5732174]
 [5.90456  ]
 [5.951708 ]
 [4.988575 ]
 [5.777872 ]
 [5.2126884]
 [5.6973   ]
 [5.5732174]
 [5.0983877]
 [5.310707 ]
 [5.2226267]
 [5.569541 ]
 [5.7737846]
 [5.7737846]
 [5.859026 ]
 [5.4277925]
 [5.4277925]
 [5.310707 ]
 [5.0386257]
 [5.4612246]
 [6.0134506]
 [5.2927275]
 [5.2927275]
 [5.5598526]
 [5.5598526]
 [5.412199 ]
 [5.412199 ]
 [5.3541775]
 [5.3541775]
 [5.570532 ]
 [5.310707 ]
 [5.586858 ]
 [5.717229 ]
 [5.4824514]
 [5.656699 ]
 [5.656699 ]
 [5.3782187]
 [5.154785 ]
 [5.310707 ]
 [5.636894 ]
 [5.568618 ]
 [5.568618 ]
 [5.656699 ]
 [5.310707 ]
 [5.310707 ]
 [5.8662004]
 [5.8747177]
 [5.5087624]
 [5.689556 ]
 [5.6969585]
 [5.932081 ]
 [5.8662004]
 [5.656699 ]
 [5.322341 ]
 [5.511917 ]
 [5.7902985]

In [40]:
min_length = min(len(smiles_list_test), len(well))
values = [val[0] for val in well[:min_length]] 

In [41]:
print(values)

[5.320606, 5.2716703, 5.059407, 5.1448517, 5.284195, 5.2422705, 5.368682, 5.3890457, 5.09416, 5.0523124, 5.008095, 5.4924116, 5.317238, 5.3535748, 5.489856, 5.3856745, 5.1945477, 4.9230795, 4.5703797, 5.1945477, 5.310707, 5.5732174, 5.90456, 5.951708, 4.988575, 5.777872, 5.2126884, 5.6973, 5.5732174, 5.0983877, 5.310707, 5.2226267, 5.569541, 5.7737846, 5.7737846, 5.859026, 5.4277925, 5.4277925, 5.310707, 5.0386257, 5.4612246, 6.0134506, 5.2927275, 5.2927275, 5.5598526, 5.5598526, 5.412199, 5.412199, 5.3541775, 5.3541775, 5.570532, 5.310707, 5.586858, 5.717229, 5.4824514, 5.656699, 5.656699, 5.3782187, 5.154785, 5.310707, 5.636894, 5.568618, 5.568618, 5.656699, 5.310707, 5.310707, 5.8662004, 5.8747177, 5.5087624, 5.689556, 5.6969585, 5.932081, 5.8662004, 5.656699, 5.322341, 5.511917, 5.7902985, 5.7984476, 5.656699, 5.8204584, 5.5533333, 5.5153084, 6.170492, 5.656699, 6.170492, 6.1144114, 5.310707, 5.8597145, 5.329629, 5.310707, 5.777872, 5.8662004, 5.777872, 5.819627, 5.3081512, 5.81073

In [43]:
# Write to CSV file
with open("mers_prediction_preliminary.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["SMILES", "MERS"])
    for smiles, value in zip(smiles_list_test, values):
        writer.writerow([smiles, value])

print("CSV file created successfully!")

CSV file created successfully!


The End.